In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [3]:
# Import restrictions
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.columns = map(str.lower, rest.columns)
rest['isbn13'] = rest['isbn13'].astype(str)
for col in ['isbn13', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn13','vendor']]
rest.head()

,isbn13,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [4]:
rest.shape

(88613, 2)

In [5]:
risky = rest[rest['vendor'] == 'RISKY']
risky.shape

(486, 2)

In [6]:
# import suppliers excel
sup_exc = pd.read_csv('suppliers_excel.csv')
sup_exc.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,max quantity,min,region
0,9780000079145,NaN,NaN,LWW,ABE,GBP,1.245501,67.0,0.55,0,0,3.47,10000,0,EUROPE
1,9780000003201,NaN,NaN,LWW,ABE,GBP,1.245501,265.0,0.55,0,0,3.47,10000,0,EUROPE
2,9780000002402,NaN,NaN,LWW,ABE,GBP,1.245501,306.0,0.55,0,0,3.47,10000,0,EUROPE
3,9780000006493,NaN,NaN,LWW,ABE,GBP,1.245501,772.0,0.55,0,0,3.47,10000,0,EUROPE
4,9780000009883,NaN,NaN,LWW,ABE,GBP,1.245501,92.0,0.55,0,0,3.47,10000,0,EUROPE


In [7]:
sup_exc['inv_price_before_exc'] = sup_exc['price']*(1-sup_exc['discount'])
sup_exc['landed_cost'] = sup_exc['inv_price_before_exc'] * sup_exc['rate'] + sup_exc['shipping cost']

In [8]:
sup_exc = sup_exc[['isbn', 'supplier', 'pub', 'currency', 'rate','price', 'discount', 'invoice_fee', 'lp_fee','inv_price_before_exc','shipping cost','landed_cost', 'max quantity']]

In [9]:
sup_sql = pd.read_csv('supplier_sql.csv')
sup_sql.head()

,isbn,supplier,pub,currency,rate,price,discount,invoice_fee,lp_fee,inv_price_before_exc,shipping cost,landed_cost,max quantity
0,9780786820771,ALEK,HACHETTE LIVE,USD,1.000000,0.01,0.59,0.005,0.0,0.004121,0.5,0.504120,5000
1,9780194705493,ALEK,OXFORD UP ELT,GBP,1.245501,0.14,0.43,0.005,0.0,0.080199,0.5,0.599888,10000
2,9780593222904,ALEK,RANDOM HOUSE,USD,1.000000,1.11,0.64,0.005,0.0,0.401598,0.5,0.901598,5000
3,9780593222898,ALEK,RANDOM HOUSE,USD,1.000000,1.11,0.64,0.005,0.0,0.401598,0.5,0.901598,5000
4,9780593100073,ALEK,RANDOM HOUSE,USD,1.000000,1.11,0.64,0.005,0.0,0.401598,0.5,0.901598,5000


In [10]:
all_sup = sup_exc.append(sup_sql, ignore_index= True)

In [11]:
all_sup.head()

,isbn,supplier,pub,currency,rate,price,discount,invoice_fee,lp_fee,inv_price_before_exc,shipping cost,landed_cost,max quantity
0,9780000079145,ABE,LWW,GBP,1.245501,67.0,0.55,0.0,0.0,30.15,3.47,41.021846,10000
1,9780000003201,ABE,LWW,GBP,1.245501,265.0,0.55,0.0,0.0,119.25,3.47,151.995957,10000
2,9780000002402,ABE,LWW,GBP,1.245501,306.0,0.55,0.0,0.0,137.70,3.47,174.975444,10000
3,9780000006493,ABE,LWW,GBP,1.245501,772.0,0.55,0.0,0.0,347.40,3.47,436.156938,10000
4,9780000009883,ABE,LWW,GBP,1.245501,92.0,0.55,0.0,0.0,41.40,3.47,55.033728,10000


In [12]:
all_sup.shape

(3895294, 13)

In [13]:
all_sup = all_sup.sort_values(by='landed_cost', ascending=True)

In [14]:
all_sup.head()

,isbn,supplier,pub,currency,rate,price,discount,invoice_fee,lp_fee,inv_price_before_exc,shipping cost,landed_cost,max quantity
574548,9780786820771,ALEK,HACHETTE LIVE,USD,1.0,0.01,0.59,0.005,0.0,0.004121,0.5,0.50412,5000
303363,9781259927638,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000
302709,9781260025415,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000
302435,9781259879906,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000
303486,9781260169829,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000


In [15]:
all_sup = pd.merge(all_sup, rest, how='left', left_on = ['isbn','supplier'], right_on = ['isbn13', 'vendor'] )
all_sup.head()


,isbn,supplier,pub,currency,rate,price,discount,invoice_fee,lp_fee,inv_price_before_exc,shipping cost,landed_cost,max quantity,isbn13,vendor
0,9780786820771,ALEK,HACHETTE LIVE,USD,1.0,0.01,0.59,0.005,0.0,0.004121,0.5,0.50412,5000,NaN,NaN
1,9781259927638,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000,NaN,NaN
2,9781260025415,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000,NaN,NaN
3,9781259879906,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000,NaN,NaN
4,9781260169829,RAJAN,MCGRAW,USD,1.0,0.01,0.48,0.000,0.0,0.005200,0.5,0.50520,10000,NaN,NaN


In [16]:
all_sup.shape

(3895294, 15)

In [17]:
all_sup.drop(
    all_sup[
        (all_sup['isbn'] == all_sup['isbn13']) & (all_sup['supplier'] == all_sup['vendor'])
        ].index, inplace = True
)

In [18]:
all_sup.shape

(3894179, 15)

In [21]:
all_sup = all_sup[~all_sup['isbn'].isin(list(risky['isbn13']))]

In [22]:
all_sup.shape

(3894007, 15)

In [23]:
all_sup.drop(columns = ['isbn13', 'vendor'], inplace = True)

In [24]:
all_sup.head()

,isbn,supplier,pub,currency,rate,price,discount,invoice_fee,lp_fee,inv_price_before_exc,shipping cost,landed_cost,max quantity
0,9780786820771,ALEK,HACHETTE LIVE,USD,1.000000,0.01,0.59,0.005,0.0,0.004121,0.5,0.504120,5000
5,9780194705493,ALEK,OXFORD UP ELT,GBP,1.245501,0.14,0.43,0.005,0.0,0.080199,0.5,0.599888,10000
6,9781259922428,RAJAN,MCGRAW,USD,1.000000,0.25,0.48,0.000,0.0,0.130000,0.5,0.630000,10000
10,9780078022159,RAJAN,MCGRAW,USD,1.000000,0.25,0.48,0.000,0.0,0.130000,0.5,0.630000,10000
23,9781259712357,RAJAN,MCGRAW,USD,1.000000,0.25,0.48,0.000,0.0,0.130000,0.5,0.630000,10000


In [26]:
all_sup['supplier'].unique()

array(['ALEK', 'RAJAN', 'FELIPE', 'BM TRADING', 'ARMANDO', 'BILLSON',
       'SENAD', 'ABE', 'ZOOKAL', 'COINFO', 'UT', 'LAURENTIU', 'BERTRAMS',
       'PROBOOK', 'ABE LAURENTIU', 'ABE ALEK', 'APEX', 'FELIPE/ARMANDO',
       'BRYAN', 'GAUTAM', 'SOHRAB', 'FRIDAY', 'BILLSON T',
       'BILLSON - SCI', 'TEXTBOOK TRADE HK', 'MYAMMAR', 'WESTERN CAMPUS',
       'WC'], dtype=object)

In [32]:
# Removing Austalian Suppliers
no_aus = all_sup[~((all_sup['supplier']=='COINFO') | (all_sup['supplier']=='ZOOKAL'))]
no_aus.shape

In [ ]:
#Dropping dupplicates

no_dup = no_aus.drop_duplicates(subset='isbn', keep="first")

In [34]:
pearson = no_dup[no_dup['pub'] == 'PEARSON']